In [48]:
# 1. Read Dataset
# 2. Extract valuable features
# 3. Data preprocessing
#   3.1 Remove strings from data
#   3.2 Informations are equal to each other
# 4. Divide dataset to train and test set
# 5. Create Neural Network
# 6. Compile and Fit
# 7. Let's predict
# 8. Congrats!

In [49]:
import pandas as pd

In [50]:
dataframe = pd.read_csv("Churn_Modelling.csv")
dataframe.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [51]:
dataframe = dataframe.iloc[:, 3:]
dataframe.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [52]:
x = dataframe.iloc[:, :-1].values
y = dataframe.iloc[:, -1].values

In [53]:
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
country_encoder = LabelEncoder()
x[:, 2] = gender_encoder.fit_transform(x[:, 2])
x[:, 1] = country_encoder.fit_transform(x[:, 1])
x[:, 1]

array([0, 2, 0, ..., 0, 1, 0], dtype=object)

In [54]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
x

array([[-0.32622142, -0.90188624, -1.09598752, ...,  0.64609167,
         0.97024255,  0.02188649],
       [-0.44003595,  1.51506738, -1.09598752, ..., -1.54776799,
         0.97024255,  0.21653375],
       [-1.53679418, -0.90188624, -1.09598752, ...,  0.64609167,
        -1.03067011,  0.2406869 ],
       ...,
       [ 0.60498839, -0.90188624, -1.09598752, ..., -1.54776799,
         0.97024255, -1.00864308],
       [ 1.25683526,  0.30659057,  0.91241915, ...,  0.64609167,
        -1.03067011, -0.12523071],
       [ 1.46377078, -0.90188624, -1.09598752, ...,  0.64609167,
        -1.03067011, -1.07636976]])

In [55]:
import torch
from torch import nn
from torch.nn import functional as F

class ArtificialNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_layer = nn.Linear(10, 32)
        self.hidden_layer1 = nn.Linear(32, 64)
        self.hidden_layer2 = nn.Linear(64, 128)
        self.hidden_layer3 = nn.Linear(128, 32)
        self.output_layer = nn.Linear(32, 1)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        x = self.hidden_layer1(x)
        x = F.relu(x)
        x = self.hidden_layer2(x)
        x = F.relu(x)
        x = self.hidden_layer3(x)
        x = F.relu(x)
        x = self.output_layer(x)
        y = F.sigmoid(x)
        
        return y
    

In [56]:
x = torch.from_numpy(x)
y = torch.from_numpy(y)
tensord_dataset = torch.utils.data.TensorDataset(x, y)


In [58]:
train_set, test_set = torch.utils.data.random_split(tensord_dataset, [int(0.8*x.shape[0]), int(0.2*x.shape[0])])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)

In [59]:
model = ArtificialNeuralNetwork()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

nb_epoch = 50
for i in range(nb_epoch):
    for data, target in train_loader:
        data = torch.autograd.Variable(data).float()
        target = torch.autograd.Variable(target).float()
        y_pred = model(data)
        loss = criterion(target, y_pred)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {i} loss: {loss.item()}")

C:\Users\marci\Anaconda3\lib\site-packages\torch\nn\functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\marci\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0 loss: 0.23227113485336304
Epoch: 1 loss: 0.22688335180282593
Epoch: 2 loss: 0.22589242458343506
Epoch: 3 loss: 0.2411920726299286
Epoch: 4 loss: 0.22732965648174286
Epoch: 5 loss: 0.22170038521289825
Epoch: 6 loss: 0.22084663808345795
Epoch: 7 loss: 0.2257978469133377
Epoch: 8 loss: 0.22471192479133606
Epoch: 9 loss: 0.21869832277297974
Epoch: 10 loss: 0.21697281301021576
Epoch: 11 loss: 0.22615765035152435
Epoch: 12 loss: 0.23241522908210754
Epoch: 13 loss: 0.2389756292104721
Epoch: 14 loss: 0.20989558100700378
Epoch: 15 loss: 0.20628930628299713
Epoch: 16 loss: 0.20646058022975922
Epoch: 17 loss: 0.23447313904762268
Epoch: 18 loss: 0.21015673875808716
Epoch: 19 loss: 0.2340044230222702
Epoch: 20 loss: 0.21727922558784485
Epoch: 21 loss: 0.19895656406879425
Epoch: 22 loss: 0.20689351856708527
Epoch: 23 loss: 0.22752252221107483
Epoch: 24 loss: 0.21277523040771484
Epoch: 25 loss: 0.2081935852766037
Epoch: 26 loss: 0.22139756381511688
Epoch: 27 loss: 0.20484963059425354
Epoch: 